# 로컬 개발에서 AWS EC2까지 Docker Compose, Git, GitHub를 사용한 Python 애플리케이션 배포 및 자동 업데이트



# 1.로컬 환경에서 Docker Compose로 Python 애플리케이션 실행

## 1.1 프로젝트 디렉토리 생성

먼저, Python 애플리케이션이 포함된 디렉토리를 생성하고 해당 디렉토리로 이동합니다.
```
mkdir stock_advisor
cd stock_advisor
```



## 1.2 Python 스크립트 작성



Python 애플리케이션 파일을 작성합니다. app.py 파일을 생성하고 아래 코드를 추가합니다.
```
app.py
```


python 코드
```python
import os
import re
import html
import requests
import pandas as pd
import gradio as gr
from dotenv import load_dotenv
from google import genai

# -----------------------------
# 1) 환경변수 로드 (.env_gemini)
# -----------------------------
load_dotenv("./.env_gemini")

USER_ID = os.getenv("user_id")
USER_SECRET = os.getenv("user_secret")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# -----------------------------
# 2) 텍스트 정제 함수
# -----------------------------
def text_clean(text: str) -> str:
    if text is None:
        return ""
    # HTML 태그 제거
    text = re.sub(r"<.*?>", "", text)
    # HTML 엔티티(&quot; 등) 복원
    text = html.unescape(text)
    return text.strip()

# -----------------------------
# 3) 네이버 뉴스 수집 함수
# -----------------------------
def fetch_naver_news(keyword: str,
                     display: int = 50,
                     max_pages: int = 2) -> pd.DataFrame:
    """
    keyword로 네이버 뉴스 검색 후 DataFrame 반환
    - display: 페이지당 결과 수 (최대 100)
    - max_pages: 가져올 최대 페이지 수
    """
    if not USER_ID or not USER_SECRET:
        raise RuntimeError("user_id / user_secret 환경변수가 설정되어 있지 않습니다.")

    url = "https://openapi.naver.com/v1/search/news"
    headers = {
        "X-Naver-Client-Id": USER_ID,
        "X-Naver-Client-Secret": USER_SECRET,
    }

    all_items = []

    # 1페이지 먼저 요청해서 total 확인
    payload = dict(query=keyword, display=display, start=1, sort="date")
    r = requests.get(url, params=payload, headers=headers)
    if r.status_code != 200:
        raise RuntimeError(f"네이버 API 오류: {r.status_code}, {r.text}")

    response = r.json()
    total = response.get("total", 0)
    if total == 0:
        return pd.DataFrame()

    total_pages = total // display + 1
    total_pages = min(total_pages, max_pages)

    all_items.extend(response.get("items", []))

    for page in range(2, total_pages + 1):
        start = (page - 1) * display + 1
        if start > 1000:  # 네이버 뉴스 API start 최대 1000
            break

        payload = dict(query=keyword, display=display, start=start, sort="date")
        r = requests.get(url, params=payload, headers=headers)
        if r.status_code != 200:
            print(f"[경고] {page}페이지 요청 실패: {r.status_code}")
            break

        resp = r.json()
        items = resp.get("items", [])
        if not items:
            break
        all_items.extend(items)

    result = {}
    for item in all_items:
        for key, value in item.items():
            if key in ["title", "description"]:
                result.setdefault(key, []).append(text_clean(value))
            else:
                result.setdefault(key, []).append(value)

    df = pd.DataFrame(result)
    return df

# -----------------------------
# 4) Gemini 요약 함수
# -----------------------------
def summarize_with_gemini(df: pd.DataFrame, keyword: str) -> str:
    if GEMINI_API_KEY is None:
        raise RuntimeError("GEMINI_API_KEY 환경변수가 설정되어 있지 않습니다.")

    client = genai.Client(api_key=GEMINI_API_KEY)

    if df.empty:
        return f"'{keyword}' 키워드로 수집된 뉴스가 없습니다."

    # 너무 길어지지 않도록 상위 20개만 사용
    df_use = df.head(20)

    news_lines = []
    for i, row in df_use.iterrows():
        title = row.get("title", "")
        desc = row.get("description", "")
        link = row.get("link", "")
        line = f"{i+1}. 제목: {title}\n   요약: {desc}\n   링크: {link}"
        news_lines.append(line)

    news_text = "\n\n".join(news_lines)

    prompt = f"""
다음은 '{keyword}' 키워드로 수집한 네이버 뉴스 목록입니다.

{news_text}

위 기사들을 바탕으로 다음 내용을 한국어로 정리해줘.

1) 전체 뉴스를 5~7줄 정도로 핵심만 요약
2) 주요 이슈/논점이 무엇인지 정리
3) 전반적인 분위기(긍정/부정/중립)를 한 줄로 평가
4) 추가로 눈에 띄는 서브 이슈가 있다면 2~3개 정도 bullet로 정리
5) 수집된 기사와 키워드의 주가를 분석해서 향후 주가에 미칠 영향 알려줘
"""

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )
    return response.text

# -----------------------------
# 5) Gradio용 파이프라인 함수
# -----------------------------
def run_pipeline(keyword: str,
                 max_pages: int = 2,
                 display: int = 50):
    keyword = keyword.strip()
    if not keyword:
        return "키워드를 입력하세요.", pd.DataFrame()

    try:
        df = fetch_naver_news(keyword, display=display, max_pages=max_pages)
    except Exception as e:
        return f"네이버 뉴스 수집 중 오류 발생:\n{e}", pd.DataFrame()

    if df.empty:
        return f"'{keyword}' 키워드로 뉴스가 없습니다.", pd.DataFrame()

    try:
        summary = summarize_with_gemini(df, keyword)
    except Exception as e:
        return f"Gemini 요약 중 오류 발생:\n{e}", df[["title", "link"]]

    # 프리뷰용으로 제목+링크만 보여줌
    preview_df = df[["title", "link"]].head(50)

    return summary, preview_df

# -----------------------------
# 6) Gradio 인터페이스 정의
# -----------------------------
with gr.Blocks(title="네이버 뉴스 + Gemini 요약") as demo:
    gr.Markdown("## 🔍 키워드 기반 네이버 뉴스 요약 서비스\n\n"
                "키워드를 입력하면 네이버 뉴스에서 기사를 가져와서 Gemini로 요약해줍니다.")

    with gr.Row():
        keyword_input = gr.Textbox(
            label="검색 키워드",
            placeholder="예) 핀테크, 인공지능, 비트코인 ..."
        )
    with gr.Row():
        max_pages_input = gr.Slider(
            minimum=1,
            maximum=5,
            value=2,
            step=1,
            label="가져올 페이지 수 (페이지당 display개)"
        )
        display_input = gr.Slider(
            minimum=10,
            maximum=100,
            value=50,
            step=10,
            label="페이지당 기사 수 (display)"
        )

    run_button = gr.Button("뉴스 수집 & 요약 실행")

    summary_output = gr.Markdown(label="Gemini 요약 결과")
    table_output = gr.Dataframe(label="수집된 뉴스 (제목 + 링크)")

    run_button.click(
        fn=run_pipeline,
        inputs=[keyword_input, max_pages_input, display_input],
        outputs=[summary_output, table_output]
    )

# -----------------------------
# 7) 실행
# -----------------------------
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860)


```



## 1.3 requirements.txt 생성

애플리케이션에서 필요한 라이브러리 목록을 requirements.txt 파일로 정의합니다.
```
requirements.txt
```

```
gradio
pandas
requests
python-dotenv
google-genai

```

## 1.4 Dockerfile 생성

Dockerfile을 작성하여 Python 3.10 이미지를 기반으로 애플리케이션을 컨테이너로 실행하도록 설정합니다.
```
Dockerfile
```

```
# 1) Python 베이스 이미지 선택
FROM python:3.10-slim

# 2) 컨테이너 안 작업 디렉토리 설정
WORKDIR /app

# 3) 시스템 필수 패키지 (선택 사항: 빌드 툴 등)
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    && rm -rf /var/lib/apt/lists/*

# 4) requirements 먼저 복사 후 설치 (캐시 활용)
COPY requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# 5) 애플리케이션 코드 복사
COPY app.py /app/app.py

# 6) Gradio가 사용할 포트 오픈
EXPOSE 7860

# 7) 컨테이너 실행 시 명령
CMD ["python", "app.py"]

```

## 1.5 docker-compose.yml 생성

docker-compose.yml 파일을 생성하여 Docker Compose로 컨테이너를 관리합니다.
```
docker-compose.yml
```

```

services:
  naver-news-gemini:
    build: .
    container_name: naver-news-gemini
    ports:
      - "7860:7860"
    # 호스트의 .env_gemini 파일을 읽어서 컨테이너 환경변수로 넣어줌
    env_file:
      - .env_gemini
    environment:
      - TZ=Asia/Seoul
    restart: unless-stopped

```

## 1.6 Docker Compose로 애플리케이션 실행

로컬에서 Docker Compose로 애플리케이션을 빌드하고 실행합니다.
```
docker-compose up --build
```
실행이 완료되면 브라우저에서 http://localhost:7860으로 접속하여 애플리케이션을 테스트할 수 있습니다.



# 2.GitHub에 프로젝트 업로드

## 2.1 GitHub에 새로운 리포지토리 생성

GitHub에서 새 리포지토리를 생성한 후, 로컬 프로젝트 디렉토리에서 Git 초기화 및 첫 번째 커밋을 진행합니다.
```
# Git 초기화
git init

# 모든 파일 추가
git add .

# 커밋
git commit -m "Initial commit"

# GitHub 리포지토리 연결
git remote add origin https://github.com/your_username/your_repository_name.git

# 원격 저장소로 푸시
git push -u origin main
```
이제 GitHub에 프로젝트가 업로드되었습니다.

# 3.EC2 인스턴스 설정 및 설치

## 3.1 EC2 인스턴스에 SSH로 접속

EC2 인스턴스에 SSH로 접속합니다.
```
ssh -i "your-key.pem" ec2-user@your-ec2-public-ip
```


## 3.2 Git 설치

EC2 인스턴스에 Git을 설치합니다.
```
sudo yum install -y git
```

## 3.3 Docker 설치

EC2 인스턴스에 Docker를 설치합니다.
```
# Docker 설치
sudo yum update -y
sudo yum install -y docker

# Docker 서비스 시작
sudo service docker start

# ec2-user에게 Docker 권한 부여
sudo usermod -a -G docker ec2-user

# 변경 사항을 적용하려면 로그아웃 후 다시 로그인
exit
```
다시 SSH로 접속합니다.
```
ssh -i "your-key.pem" ec2-user@your-ec2-public-ip
```
Docker가 정상적으로 설치되었는지 확인합니다.
```
docker --version
```

## 3.4 Docker Compose 설치

EC2 인스턴스에 Docker Compose를 설치합니다.
```
sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
```
실행 권한 부여
```
sudo chmod +x /usr/local/bin/docker-compose
```
Docker Compose 버전 확인
```
docker-compose --version
```


# 3.5 Docker Buildx 설치 0.17이상 반드시 필요

1) CLI 플러그인 디렉토리 생성
```bash
mkdir -p ~/.docker/cli-plugins/
```

2) Buildx 최신 버전 다운로드
```bash
curl -SL https://github.com/docker/buildx/releases/download/v0.17.0/buildx-v0.17.0.linux-amd64 \
  -o ~/.docker/cli-plugins/docker-buildx

```

3) 실행 권한 부여
```bash
chmod +x ~/.docker/cli-plugins/docker-buildx
```

4) 버전 확인 (0.17.x 이상 나오면 OK)
```bash
docker buildx version
```

# 4.EC2 인스턴스에서 프로젝트 가져오기 및 실행

## 4.1 GitHub 리포지토리 클론

EC2 인스턴스에서 GitHub 리포지토리를 클론합니다.<br>
GitHub 리포지토리 클론
```
git clone https://github.com/your_username/your_repository_name.git
```
클론한 디렉토리로 이동
```
cd your_repository_name
```

## 4.2 Docker Compose 실행
EC2에서 Docker Compose로 애플리케이션을 빌드하고 실행합니다.<br>
Docker Compose 빌드 및 실행
```
docker-compose up --build
```
이제 EC2 인스턴스에서 애플리케이션이 실행됩니다. 인스턴스의 퍼블릭 IP로 접속하여 애플리케이션에 접근할 수 있습니다.

예: http://your-ec2-public-ip:7860

# 5.GitHub Actions을 사용한 1주일에 한 번 자동 업데이트

## 5.1 GitHub Secrets 설정
먼저, EC2 인스턴스에 대한 SSH 접속을 위해 GitHub Secrets에 SSH 비밀번호 또는 SSH 키를 설정해야 합니다.<br>
GitHub Repository Settings로 이동합니다.<br>
좌측 메뉴에서 Secrets and variables -> Actions로 이동한 후 New repository secret을 클릭합니다.<br>
아래와 같은 항목들을 추가합니다:<br>
AWS_HOST: EC2 퍼블릭 IP (예: 43.202.33.102)<br>
AWS_USER: EC2 접속 사용자 (예: ec2-user)<br>
AWS_KEY: SSH 비밀 키 (.pem 파일 내용)<br>


## 5.2 GitHub Actions 설정

프로젝트 루트 디렉토리에 .github/workflows 디렉토리를 생성하고, 그 안에 deploy.yml 파일을 만듭니다.
```
mkdir -p .github/workflows
nano .github/workflows/deploy.yml
```
deploy.yml 파일에 아래 내용을 추가합니다
```
name: Deploy to EC2

on:
  schedule:
    - cron: '0 0 * * 0' # 매주 일요일 00:00에 실행

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout code
      uses: actions/checkout@v2

    - name: Connect to EC2 and update service
      uses: appleboy/ssh-action@v0.1.3
      with:
        host: ${{ secrets.AWS_HOST }}
        username: ${{ secrets.AWS_USER }}
        key: ${{ secrets.AWS_KEY }}
        script: |
          cd your_repository_name  # 프로젝트 디렉토리로 이동
          git pull origin master  # 최신 코드 가져오기
          docker-compose down  # 기존 컨테이너 중지
          docker-compose up --build -d  # 새로운 컨테이너 빌드 및 실행
```
이 deploy.yml 파일은 GitHub Actions가 설정된 일정(매주 일요일 00:00)에 EC2 서버에 SSH로 연결하여 최신 코드를 가져오고, Docker 컨테이너를 재빌드하여 배포하는 과정을 자동으로 수행합니다.

# 6.서비스 테스트 및 관리

## 6.1 서비스 확인

EC2에서 실행 중인 Docker 컨테이너가 잘 동작하는지 확인하려면 웹 브라우저에서 EC2 인스턴스의 퍼블릭 IP와 포트 7860으로 접속합니다.<br>

예: http://your-ec2-public-ip:7860
이 주소로 접속하여 Gradio 인터페이스가 정상적으로 작동하는지 확인할 수 있습니다.

## 6.2 서비스 중지

서비스를 중지하려면 Docker Compose를 중지합니다.
```
docker-compose down
```
이 명령어를 실행하면 실행 중인 컨테이너가 중지되고 제거됩니다.

## 6.3 백그라운드에서 실행

Docker Compose를 백그라운드에서 실행하려면 -d 옵션을 사용합니다.
```
docker-compose up -d --build
```
이 명령어를 실행하면 Docker가 백그라운드에서 애플리케이션을 실행하게 되며, EC2 인스턴스를 다시 시작하거나 연결을 끊어도 애플리케이션이 계속 작동합니다.